In [ ]:
import pandas as pd
import os
import warnings
import seaborn as sns

warnings.filterwarnings('ignore')

## Data Input

1. Buat folder **import**
2. Masukkan file data yang ingin diinput ke dalam folder **import**
3. Kita akan memasukkan semua data pada folder **import** ke dalam satu **DataFrame**
   **DataFrame** adalah objek tabel dalam pandas-python ini serupa dengan spreadsheet pada excel

In [ ]:
data = pd.DataFrame()

for file in os.listdir("import"):
    file_location = "import/"+file
    tables = pd.read_html(file_location, header=1, converters={"Kode Destinasi": str})
    data = pd.concat([data, tables[0]])
data

## Data Preprocessing
Data preprocessing yang akan kita lakukan adalah
1. Mengambil Kolom data yang diperlukan saja
   Kolom yang akan kita ambil **["Kode Destinasi","Id Order", "Tanggal", "Pcs", "Nominal Transaksi", "Detail Pesanan", "Nama Penerima", "Alamat Lengkap"]**
2. Rename nama kolom
3. Mengubah tipe data
4. Membersihkan format nama
5. Membersihkan format nomor HP
6. Membersihkan data item
7. Membersihkan data pcs
8. Membersihkan data grand total
9. Set phone number sebagai customer id

### Mengambil kolom data yang diperlukan

In [ ]:
data = data[["Kode Destinasi","Id Order", "Tanggal", "Pcs", "Nominal Transaksi", "Detail Pesanan", "Nama Penerima", "Alamat Lengkap"]]
data

### Rename nama kolom

In [ ]:
data = data.rename(columns={
    "Kode Destinasi": "phone", 
    "Tanggal": "order_date", 
    "Id Order":"order_id",
    "Pcs": "order_size",
    "Nominal Transaksi":"grand_total",
    "Detail Pesanan": "item",
    "Nama Penerima": "name",
    "Alamat Lengkap": "address"})

data

### Mengubah tipe data

In [ ]:
retention_data = data

retention_data["order_date"] = pd.to_datetime(retention_data["order_date"], dayfirst=True).dt.date
retention_data['order_date'] = pd.to_datetime(retention_data['order_date'])

retention_data

### Membersihkan format Nama

In [ ]:
import re

def name_cleansing(x):
    x = x.lower()
    list_of_title = [
        "ibu ", "bu ", "hj ", "h ", "bpk ", "pak ", "pa ", "bapak ", "hj. ", "h. ", "a ", "bpk. ", "l ", "lik ", "i ", "wayan ", "m ", "m."
    ]
    regex = re.compile('|'.join(map(re.escape, list_of_title)))
    result = regex.sub("", str(x))
    
    return result.lstrip().title() 

retention_data["clean_name"] = retention_data["name"].str.replace('^\d+\s+', '')
retention_data["clean_name"] = retention_data["clean_name"].apply(lambda x: name_cleansing(str(x)))
retention_data

### Membersihkan format nomor HP

In [ ]:
def normalize_phone(x):
    if x[0:1] == "+":
        return x[1:]
    if x[0:1] == "0":
        return "62"+x[1:]
    if x[0:1] == "8":
        return "62"+x
    return x

retention_data["normalized_phone"] = retention_data["phone"].apply(lambda x: normalize_phone(str(x)))
retention_data

### Membersihkan data item

In [ ]:
retention_data["item"] = retention_data["item"].str.replace('^\d+\s+', '') ## replace numeric dengan empty string
retention_data

### Membersihkan data pcs

In [ ]:
retention_data["order_size"] = retention_data["order_size"].str.replace('[^0-9]', '') ## replace non numeric dengan empty string
retention_data

In [ ]:
retention_data["grand_total"] = retention_data["grand_total"].str.replace('[^0-9]', '').astype(int)
retention_data

### Set Phone Number sebagai customer id

In [ ]:
retention_data["customer"] = retention_data["phone"].apply(lambda x: normalize_phone(str(x)))
retention_data

## Data Processing

Pada data processing kali ini kita akan membuat RFM Tabel dan RFM Class yang perlu dilakukan adalah
1. Membuat RFM tabel menggunakan data agregat
   a. max order date sebagai recency
   b. banyaknya order sebagai frequency
   c. jumlah grand_total sebagai monetary
2. Membuat klasifikasi RFM
    

### Membuat RFM Tabel

In [ ]:
import datetime as dt
NOW = retention_data["order_date"].max() ### ambil tanggal order terakhir sebagai tanggal hari ini

rfmTable = retention_data.groupby('customer').agg({'order_date': lambda x: (x.max() - NOW).days, # Recency
                                        'order_id': lambda x: len(x),      # Frequency
                                        'grand_total': lambda x: x.sum()}) # Monetary Value

rfmTable['order_date'] = rfmTable['order_date'].astype(int)
rfmTable.rename(columns={'order_date': 'recency', 
                         'order_id': 'frequency', 
                         'grand_total': 'monetary_value'}, inplace=True)

rfmTable.sort_values(by=["frequency"], ascending=False).head() ## check rfmTable

### Membuat klasifikasi RFM
Kita akan membuat kategori/kelas untuk tiap attribut recency, frequency, dan monetery_value.
Metode yang digunakan untuk kelas recency dan monetery value adalah dengan menggunakan kuantil. Kita akan membagi data recency dan monetery ke dalam 5 kelas berdasarkan kuantil.

Sedangkan untuk frequency kita akan menggunakan ketegorisasi seperti ini
1. Jika frequency pembelian adalah 1 maka kita beri dia kelas 1
2. Jika frequency pembelian adalah 2 maka kita beri dia kelas 2
3. Jika frequency pembelian adalah 3 maka kita beri dia kelas 3
4. Jika frequency pembelian adalah 4 maka kita beri dia kelas 4
5. Jika frequency pembelian adalah 5 atau lebih maka kita beri dia kelas 5

In [ ]:
def FClassMap(x):
    class_map = {
    1: 1,
    2: 2,
    3: 3,
    4: 4,
    "other": 5 
}
    if x in class_map:
        return class_map[x]
    else:
        return class_map["other"]

rfmSegmentation = rfmTable.copy()
rfmSegmentation['R_Class'] = pd.qcut(rfmSegmentation['recency'], q=5, labels=[1,2,3,4,5])
rfmSegmentation['F_Class'] = rfmSegmentation['frequency'].apply(FClassMap)
rfmSegmentation['M_Class'] = pd.qcut(rfmSegmentation['monetary_value'], q=5, labels=[1,2,3,4,5])

rfmSegmentation

### Membuat kelas RFM
Kelas RFM ini adalah gabungan dari kategori Recency + kategori Frequency + kategori Monetary

In [ ]:
rfmSegmentation['RFMClass'] = rfmSegmentation.R_Class.astype(str) \
                           + rfmSegmentation.F_Class.astype(str) \
                            + rfmSegmentation.M_Class.astype(str)
rfmSegmentation

## Data Visualization

Kita akan mencoba membuat visualisasi data dari hasil pengolahan data di atas


### Scatter plot 

kita akan coba melihat persebaran data berdasarkan recency, frequency, dan monetary value menggunakan scatter plot

In [ ]:
sns.scatterplot(data=rfmSegmentation, x='recency', y='frequency', hue='monetary_value')

### heat map kategori RFM

In [ ]:
plotdata = rfmSegmentation.groupby(['F_Class', 'R_Class']).agg(['sum','count', 'mean'])['monetary_value']

ax = sns.scatterplot(data=plotdata, x='R_Class', y='F_Class', size=plotdata['mean'].fillna(0), hue=plotdata['count'], sizes=(100, 400))
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))

## Filtering & Ekspor Data

Pada bagian kali ini kita akan mencoba melakukan filtering dan ekspor data ke file exce.
1. Kita mau ambil data dengan rfm class 555
2. Ekspor ke file excel

### Filter Data

In [ ]:
filtered_data = rfmSegmentation.query("RFMClass == '555'")
filtered_data

### Export to excel

In [ ]:
filtered_data.to_excel("export/data_export.xlsx", index=False)